In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.offline import plot, iplot, init_notebook_mode
init_notebook_mode(connected=True)
%matplotlib inline

# Wczytywanie danych

In [ ]:
! pip install kaggle

In [2]:
df = pd.read_csv("PS_20174392719_1491204439457_log.csv")

ParserError: ignored

In [ ]:
df.info()

In [ ]:
{column: len(df[column].unique()) for column in df.columns}

In [ ]:
paysim_pivot1=pd.pivot_table(df,index=["type"],
                               values=['amount','isFraud','isFlaggedFraud'],
                               aggfunc=[np.sum,np.std], margins=True)

#Adding color gradient 
#cm = sns.light_palette("blue", as_cmap=True)
paysim_pivot1.style.background_gradient(cmap=cm)

In [ ]:
paysim_pivot2=pd.pivot_table(df,index=["type"],
                               values=['amount','oldbalanceOrg','newbalanceOrig'],
                               aggfunc=[np.sum], margins=True)

#Adding style
#paysim_pivot2.style\
#    .format('{:.2f}')\
#    .bar(align='mid',color=['darkred'])\
#   .set_properties(padding='5px',border='3px solid white',width='200px')
paysim_pivot2.style.background_gradient(cmap=cm)

In [ ]:
paysim_pivot3=pd.pivot_table(df,index=["type"],
                               values=['amount','oldbalanceDest','newbalanceDest'],
                               aggfunc=[np.sum], margins=True)

#Adding style
paysim_pivot3.style.background_gradient(cmap=cm)

In [ ]:
fig = px.box(df, y="amount", nrows=50000)
fig.show()

In [ ]:
df.iloc[:]

In [ ]:
df.describe()

1) Najpierw szukamy rodzajów płatności, kiedy stwierdzono oszustwo.

In [ ]:
print('\n Rodzaje płatności, kiedy transakcje okazały się oszustwem to: {}'.format(\
list(df.loc[df.isFraud == 1].type.drop_duplicates().values)))

Ile z tych transakcji to były płatności przelewem 'TRANSFER', a ile to 'CASH_OUT'?

In [ ]:
dfFraudByTransfer = df.loc[(df.isFraud == 1)  & (df.type == 'TRANSFER')]
print("Liczba  oszukańczych przelewów: {}".format(len(dfFraudByTransfer)))

In [ ]:
dfFraudByCashOut = df.loc[(df.isFraud == 1)  & (df.type == 'CASH_OUT')]
print("Liczba  oszukańczych wypłat gotówki: {}".format(len(dfFraudByCashOut)))

2) Sprawdzamy, co implikuje, że atrybut isFlaggedFraud jest ustawiony, a co nie.

In [ ]:
print('\n Rodzaje transakcji, gdzie atrybut isFlaggedFraud jest ustawiony: \
{}'.format(list(df.loc[df.isFlaggedFraud == 1].type.drop_duplicates()))) 
dfTransferIsFlaggedFraud = df.loc[(df.type == 'TRANSFER') & (df.isFlaggedFraud == 1)]
print("\n liczba transakcji jako TRANSFER, gdzie atrybut isFlaggedFraud jest aktywny: {}".format(len(dfTransferIsFlaggedFraud)))

Daje nam to informacje, że tylko dla wartości "TRANSFER" jest atrybut isFlaggedFraud ustawiony i jest takich 16 transakcji. Sprawdźmy zatem, jaka jest minimalna kwota transakcji, dla której atrybut isFlaggedFraud jest aktywny.

In [ ]:
dfTransfer = df.loc[df.type == 'TRANSFER']
dfFlagged = df.loc[df.isFlaggedFraud == 1]
dfNotFlagged = df.loc[df.isFlaggedFraud == 0]

In [ ]:
print("\n Minimalna kwota transakcji dla której atrybut isFlaggedFraud ustawiony: {}".format(dfFlagged.amount.min()))

In [ ]:
print("\n Maksymalna kwota transakcji dla TRANSFER, kiedy isFlaggedFraud jest nieaktywny: {}".
     format(dfNotFlagged.amount.max()))

Okazuje się, że flaga detekcji próby nielegalnych transakcji nie daje zbyt jasnych wyników, ponieważ występuje tylko 16 razy w bazie danych ponad 6 milionów rekordów. Ponadto jest ustawiona dosyć nisko, ponieważ "zapala się" dopiero przy próbie transakcji kwoty rzędu 350 tysięcy. Natomiast ciekawa jest sytuacja, w której przy przelewie na kwotę ponad 9 milionów atrybut isFlaggedFraud jest nieaktywny.

Sprawdzamy, czy atrybuty oldBalanceDest i newBalanceDest wpływają na ustawienie atrybutu isFlaggedFraud.

In [ ]:
dfDiffOldAndNewBalanceDest = dfTransfer.loc[(dfTransfer.isFlaggedFraud == 0) & (dfTransfer.oldbalanceDest == 0) &
                                           (dfTransfer.newbalanceDest == 0)]
print("\nLiczba transakcji typu TRANSFER gdzie stare saldo=0, nowe saldo=0 i isFlaggedFraud nieaktywny: {}"
      .format(len(dfDiffOldAndNewBalanceDest)))

In [ ]:
dfTransfer.loc[1:1000]